In [52]:
import multiprocessing
import psycopg2  # For connecting to PostgreSQL database
import pandas as pd # Data analysis toolkit with flexible data structures
import numpy as np # Fundamental toolkit for scientific computation with N-dimensional array support
from sklearn.mixture import GaussianMixture # Gaussian Mixutre Model in scikit-learn
from sqlalchemy import create_engine
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

In [2]:
conn = psycopg2.connect("dbname='yelp' host='' user='' password=''")
cur = conn.cursor()

In [ ]:
query1 = "select category, count(distinct business_id) as business_count from business_category group by category order by business_count desc;"

In [ ]:
cur.execute(query1)
data = cur.fetchall()

In [ ]:
df1 = pd.DataFrame(data) 
df1.head()

In [ ]:
df1.rename(columns={df1.columns[0]: 'category', df1.columns[1]: 'business_count'}, inplace=True)
df1.head()

In [ ]:
query2 ="select bc.business_id, bc.category, r.review_id, r.user_id from business_category as bc join review as r on bc.business_id = r.business_id order by bc.business_id;"

In [ ]:
cur.execute(query2)
data = cur.fetchall()

In [ ]:
df2 = pd.DataFrame(data) 
df2.head()

In [ ]:
df2.rename(columns={df2.columns[0]: 'business_id', df2.columns[1]: 'category', df2.columns[2]: 'review_id', df2.columns[3]: 'user_id'}, inplace=True)
df2.head()

In [ ]:
query3 = "select t1.user_id, count(distinct t1.business_id) as review_count from (select bc.business_id, bc.category, r.review_id, r.user_id from business_category as bc join review as r on bc.business_id = r.business_id where bc.category like 'Food' order by bc.business_id) as t1 group by t1.user_id order by review_count desc limit 1000;"
cur.execute(query3)
data = cur.fetchall()
df3 = pd.DataFrame(data) 
df3.rename(columns={df3.columns[0]: 'user_id', df3.columns[1]: 'review_count'}, inplace=True)
df3.head()

In [ ]:
df3['reviewed_percentage'] = df3['review_count']*100/21189
df3.head()

In [ ]:
# Frequency table for review count/year for users

import psycopg2
import datetime

db_conn = psycopg2.connect("dbname='yelp' host='manoj7010xgvd.ddns.net' user='manoj' password='^&YTGXXkhashfghs)('")
cur = db_conn.cursor()
cur.execute("select review_count, yelping_since from _user;")
data = cur.fetchall()

#print(len(data))
#print(data)

current_date = datetime.datetime.now().date()

#print (type(data), '\n', data,'\n')

review_frquency = []


for item in data:
    # Assumption is every user is still active
    days_active = (current_date - item[1]).days
    reviews_per_day = item[0]/days_active
    reviews_per_year = reviews_per_day*365
    review_frquency.append(reviews_per_year)
    #print (item[1], '\t', days_active, '\t', item[0], '\t', reviews_per_year)

#print(review_frquency)

frequency = []

for i in range(0,200):
     frequency.append(0)

#print(frequency)


for review in review_frquency:
    #print (review)
    count = (review//10)
    #print(int(count))
    frequency[int(count)] = frequency[int(count)] + 1
#
print(frequency)


cur.close()
db_conn.close()


In [ ]:
query4 = "select t1.user_id, count(distinct t1.business_id) as review_count from (select bc.business_id, bc.category, r.review_id, r.user_id from business_category as bc join review as r on bc.business_id = r.business_id where bc.category like 'Nightlife' order by bc.business_id) as t1 group by t1.user_id order by review_count desc;"
cur.execute(query4)
data = cur.fetchall()
df4 = pd.DataFrame(data) 
df4.rename(columns={df4.columns[0]: 'user_id', df4.columns[1]: 'review_count'}, inplace=True)
df4.head()

In [ ]:
df4['reviewed_percentage'] = df4['review_count']*100/10524
df4.head()

In [ ]:
df4.to_csv('nightlife.csv')

In [ ]:
word_count_query = "select bc.business_id, bc.category, r.review_id, r.user_id, length(r.review_text) as review_length from business_category as bc join review as r on bc.business_id = r.business_id where bc.category like 'Food' order by bc.business_id limit 100;"
cur.execute(word_count_query)
data = cur.fetchall()
df5 = pd.DataFrame(data)
df5.head()

In [ ]:
print(df5[4].mean(), df5[4].std())

In [3]:
topic_authority = "select t1.user_id, count(distinct t1.review_id) as review_count, sum(t1.review_length) from (select bc.business_id, r.review_id, r.user_id, length(r.review_text) as review_length from business_category as bc join review as r on bc.business_id = r.business_id where bc.category like 'Food' order by bc.business_id) as t1 group by t1.user_id order by review_count desc limit 1000;"
cur.execute(topic_authority)
data = cur.fetchall()
df6 = pd.DataFrame(data)
df6.head()

,0,1,2
0,CxDOIDnH8gp9KXzpBHJYXw,1123,792859
1,bLbSNkLggFnqwNNzzq-Ijw,552,568725
2,U4INQZOPSUaj8hMjLlZ3KA,363,777775
3,PKEzKWv_FktMm2mGPjwd0Q,362,342364
4,DK57YibC5ShBmqQl97CKog,331,275706


In [4]:
df6.rename(columns={df6.columns[0]: 'user_id', df6.columns[1]: 'review_count', df6.columns[2]: 'review_length'}, inplace=True)
df6['reviewed_percentage'] = df6['review_count']*100/21189
df6['avg_review_len'] = df6['review_length']/df6['review_count']
df6.head()

,user_id,review_count,review_length,reviewed_percentage,avg_review_len
0,CxDOIDnH8gp9KXzpBHJYXw,1123,792859,5.299920,706.018700
1,bLbSNkLggFnqwNNzzq-Ijw,552,568725,2.605125,1030.298913
2,U4INQZOPSUaj8hMjLlZ3KA,363,777775,1.713153,2142.630854
3,PKEzKWv_FktMm2mGPjwd0Q,362,342364,1.708434,945.756906
4,DK57YibC5ShBmqQl97CKog,331,275706,1.562131,832.948640


In [5]:
df6['topic_authority'] = (df6['reviewed_percentage'] > 0.5) & (df6['avg_review_len'] > 560.23)

In [6]:
df6

,user_id,review_count,review_length,reviewed_percentage,avg_review_len,topic_authority
0,CxDOIDnH8gp9KXzpBHJYXw,1123,792859,5.299920,706.018700,True
1,bLbSNkLggFnqwNNzzq-Ijw,552,568725,2.605125,1030.298913,True
2,U4INQZOPSUaj8hMjLlZ3KA,363,777775,1.713153,2142.630854,True
3,PKEzKWv_FktMm2mGPjwd0Q,362,342364,1.708434,945.756906,True
4,DK57YibC5ShBmqQl97CKog,331,275706,1.562131,832.948640,True
5,d_TBs6J3twMy9GChqUEXkg,301,275422,1.420548,915.023256,True
6,gwIqbXEXijQNgdESVc07hg,276,245678,1.302563,890.137681,True
7,UYcmGbelzRa0Q6JqzLoguw,269,208973,1.269527,776.851301,True
8,cMEtAiW60I5wE_vLfTxoJQ,266,90571,1.255368,340.492481,False
9,Wu0yySWcHQ5tZ_59HNiamg,265,284888,1.250649,1075.049057,True


In [7]:
train, test = train_test_split(df6, test_size = 0.3)
print('Training samples', len(train))
print('Testing samples', len(test))

Training samples 700
Testing samples 300


In [8]:
#http://stackoverflow.com/questions/29221502/pandas-selecting-discontinuous-columns-from-a-dataframe
features = df6[['review_count', 'avg_review_len']]
#features.head()
#clf = GaussianNB()
print(features)

     review_count  avg_review_len
0            1123      706.018700
1             552     1030.298913
2             363     2142.630854
3             362      945.756906
4             331      832.948640
5             301      915.023256
6             276      890.137681
7             269      776.851301
8             266      340.492481
9             265     1075.049057
10            261     1065.977011
11            259      994.027027
12            258      682.631783
13            241      609.099585
14            238     1404.890756
15            225     1065.582222
16            216      709.967593
17            216     1172.564815
18            214      731.630841
19            204      341.161765
20            198      747.848485
21            196      942.903061
22            194     1037.314433
23            193     1029.471503
24            191      524.277487
25            186     1218.408602
26            185     1552.821622
27            182     2075.846154
28            

In [62]:
#clf = linear_model.LogisticRegression()
#clf = RandomForestClassifier(n_jobs=8)
clf = GaussianNB()
#y, _ = pd.factorize(train['species'])
#y = df[9]
#y = np.asarray(train[df[10]])
#
y = pd.factorize(train['topic_authority'])[0]
#y = np.asarray(train['topic_authority'], dtype="bool")
#train[features]
#train[['review_count', 'avg_review_len']]

#clf.fit(train[['review_count', 'reviewed_percentage', 'avg_review_len']], y)
clf.fit(train[['review_count', 'avg_review_len']], y)

GaussianNB(priors=None)

In [21]:
pred_prob = clf.predict_proba(test[['review_count', 'reviewed_percentage', 'avg_review_len']].values)
print(pred_prob)

[[  9.71183619e-01   2.88163806e-02]
 [  9.88472645e-01   1.15273550e-02]
 [  9.74010887e-01   2.59891131e-02]
 [  9.19588598e-01   8.04114022e-02]
 [  9.22095613e-01   7.79043870e-02]
 [  9.81319874e-01   1.86801260e-02]
 [  9.55226180e-01   4.47738199e-02]
 [  9.45686572e-01   5.43134275e-02]
 [  9.86873270e-01   1.31267299e-02]
 [  4.94115511e-01   5.05884489e-01]
 [  9.85462017e-01   1.45379826e-02]
 [  8.78770970e-01   1.21229030e-01]
 [  9.88233411e-01   1.17665886e-02]
 [  9.45691546e-01   5.43084543e-02]
 [  9.54343221e-01   4.56567789e-02]
 [  9.44236508e-01   5.57634924e-02]
 [  9.10907046e-01   8.90929543e-02]
 [  9.41387058e-01   5.86129418e-02]
 [  5.57963484e-01   4.42036516e-01]
 [  9.85259450e-01   1.47405499e-02]
 [  9.60330383e-01   3.96696166e-02]
 [  9.87075167e-01   1.29248334e-02]
 [  9.70457992e-01   2.95420077e-02]
 [  9.54289594e-01   4.57104060e-02]
 [  9.68367339e-01   3.16326614e-02]
 [  9.54424195e-01   4.55758050e-02]
 [  9.85581356e-01   1.44186437e-02]
 

In [22]:
pred_prob[:,1]

array([ 0.02881638,  0.01152736,  0.02598911,  0.0804114 ,  0.07790439,
        0.01868013,  0.04477382,  0.05431343,  0.01312673,  0.50588449,
        0.01453798,  0.12122903,  0.01176659,  0.05430845,  0.04565678,
        0.05576349,  0.08909295,  0.05861294,  0.44203652,  0.01474055,
        0.03966962,  0.01292483,  0.02954201,  0.04571041,  0.03163266,
        0.04557581,  0.01441864,  0.01117984,  0.93293926,  0.01149462,
        0.05112731,  0.11031273,  0.01285929,  0.1395918 ,  0.01613419,
        0.11258609,  0.01373471,  0.02168291,  0.01207058,  0.02686841,
        0.00997771,  0.01812161,  0.03663211,  0.40810729,  0.01377189,
        0.02380574,  0.01227102,  0.01714154,  0.019867  ,  0.0118059 ,
        0.02005899,  0.01074763,  0.09357383,  0.03804014,  0.17211197,
        0.01411969,  0.04969523,  0.01424243,  0.0124109 ,  0.07894333,
        0.0118132 ,  0.64420574,  0.07886378,  0.02986717,  0.11938637,
        0.33963351,  0.01877908,  0.01961183,  0.01211209,  0.03

In [23]:
test[['review_count', 'reviewed_percentage', 'avg_review_len']]

,review_count,reviewed_percentage,avg_review_len
470,61,0.287885,630.967213
910,43,0.202935,781.883721
671,52,0.245410,2111.942308
264,79,0.372835,1052.506329
275,78,0.368116,1129.012821
717,50,0.235971,1217.320000
358,70,0.330360,517.771429
322,73,0.344518,677.835616
830,46,0.217094,662.847826
70,128,0.604087,322.664062


In [63]:
#df = pd.DataFrame(clf.predict_proba(test[['review_count', 'reviewed_percentage', 'avg_review_len']]))
df = pd.DataFrame(clf.predict_proba(test[['review_count', 'avg_review_len']]))
df

,0,1
0,9.907067e-01,0.009293
1,9.870527e-01,0.012947
2,9.871006e-01,0.012899
3,9.653105e-01,0.034689
4,9.644219e-01,0.035578
5,9.795433e-01,0.020457
6,9.910671e-01,0.008933
7,9.855036e-01,0.014496
8,9.902733e-01,0.009727
9,4.585253e-01,0.541475


In [64]:
d = test.reset_index()
d

,index,user_id,review_count,review_length,reviewed_percentage,avg_review_len,topic_authority
0,470,aNmrSw98PhdvZkLgxP2PWg,61,38489,0.287885,630.967213,False
1,910,UL5K2rnSYIPD1LcqPgbmDQ,43,33621,0.202935,781.883721,False
2,671,AwyM03qzhwgyskBruurWQw,52,109821,0.245410,2111.942308,False
3,264,ACwN3HGetqPVhAgGKzdnsA,79,83148,0.372835,1052.506329,False
4,275,KPY6FvT6h-r0W0HAC1Buag,78,88063,0.368116,1129.012821,False
5,717,r-zUIQPaHzvIyL93wQaoiQ,50,60866,0.235971,1217.320000,False
6,358,_oCiEDUZqu91P0Tzdl_wMA,70,36244,0.330360,517.771429,False
7,322,Nvpx9fUIAB3oddqgRLEPfg,73,49482,0.344518,677.835616,False
8,830,i7mBC7m7k2FLrdVUx0UqUg,46,30491,0.217094,662.847826,False
9,70,S9Jw00eZHVj5_0sOM_C5Rg,128,41301,0.604087,322.664062,False


In [66]:
df[['user_id','review_count','review_length','reviewed_percentage','avg_review_len','topic_authority']] = d[['user_id','review_count','review_length','reviewed_percentage','avg_review_len','topic_authority']]

In [67]:
#temp = pd.DataFrame(clf.predict(test[['review_count', 'reviewed_percentage', 'avg_review_len']]))
temp = pd.DataFrame(clf.predict(test[['review_count', 'avg_review_len']]))
temp

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,1


In [68]:
df['topic_authority_label'] = temp[0]
df

,0,1,user_id,review_count,review_length,reviewed_percentage,avg_review_len,topic_authority,topic_authority_label
0,9.907067e-01,0.009293,aNmrSw98PhdvZkLgxP2PWg,61,38489,0.287885,630.967213,False,0
1,9.870527e-01,0.012947,UL5K2rnSYIPD1LcqPgbmDQ,43,33621,0.202935,781.883721,False,0
2,9.871006e-01,0.012899,AwyM03qzhwgyskBruurWQw,52,109821,0.245410,2111.942308,False,0
3,9.653105e-01,0.034689,ACwN3HGetqPVhAgGKzdnsA,79,83148,0.372835,1052.506329,False,0
4,9.644219e-01,0.035578,KPY6FvT6h-r0W0HAC1Buag,78,88063,0.368116,1129.012821,False,0
5,9.795433e-01,0.020457,r-zUIQPaHzvIyL93wQaoiQ,50,60866,0.235971,1217.320000,False,0
6,9.910671e-01,0.008933,_oCiEDUZqu91P0Tzdl_wMA,70,36244,0.330360,517.771429,False,0
7,9.855036e-01,0.014496,Nvpx9fUIAB3oddqgRLEPfg,73,49482,0.344518,677.835616,False,0
8,9.902733e-01,0.009727,i7mBC7m7k2FLrdVUx0UqUg,46,30491,0.217094,662.847826,False,0
9,4.585253e-01,0.541475,S9Jw00eZHVj5_0sOM_C5Rg,128,41301,0.604087,322.664062,False,1


In [69]:
z = pd.factorize(test['topic_authority'])[0]
ari = adjusted_rand_score(z, df['topic_authority_label'].values)
print("ARI = ", ari)

ARI =  0.733653244904


ARI =  0.605472726715 with 'review_count', 'avg_review_len'
ARI =  0.325544970819 with 'reviewed_percentage', 'avg_review_len'

RANDOM FOREST
ARI =  1.0 with 'reviewed_percentage', 'avg_review_len'

In [ ]:
cur.close()
conn.close()